In [ ]:
import numpy as np
import dask.array as da
from dask.diagnostics import ProgressBar

In [ ]:
rng = da.random.default_rng()
X = rng.standard_normal((4000, 3, 500, 500), chunks=(-1, -1, 25, 50))
X

In [ ]:
%run ../TC/EC_function.ipynb

In [ ]:
dask_graph = X.map_blocks(lambda X: ec_covar_multi(X, corr_sets=[1,2,3])[0], dtype=np.float64,
                          chunks=(X.chunksize[1], X.chunksize[1]) + X.chunksize[2:],
                          drop_axis=[0, 1], new_axis=[0, 1])
dask_graph

In [ ]:
# dask_graph.visualize()

In [ ]:
with ProgressBar():
    covar_dask = dask_graph.compute()